# Synthesize a Relational Database from a SQL Schema with PluRel

This notebook shows the minimal end-to-end flow:

1. Write a small `schema.sql` file (two tables + FK constraint).
2. Configure PluRel synthesis parameters.
3. Generate a synthetic database.
4. Inspect the resulting tables.

> **Note:**
- This notebook assumes `plurel` is installed and importable in your environment.
- text columns (other than enums) are not supported yet.

## Prerequisites

In [1]:
# Ensure proper imports
import plurel

## 1. Create `schema.sql`

In [2]:
from pathlib import Path

schema_sql = """CREATE TABLE users (
    user_id BIGINT PRIMARY KEY,
    status TEXT CHECK (status IN ('active', 'inactive', 'banned'))
);

CREATE TABLE orders (
    order_id BIGINT PRIMARY KEY,
    user_id BIGINT,
    amount DOUBLE,
    order_type TEXT CHECK (order_type IN ('online', 'instore')),
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
"""

schema_path = Path("schema.sql")
schema_path.write_text(schema_sql)
print(f"Wrote schema to: {schema_path.resolve()}")

Wrote schema to: /Users/vigneshk/plurel/examples/schema.sql


## 2. Configure synthesis

In [3]:
from plurel.dataset import SyntheticDataset
from plurel.config import Config, DatabaseParams, Choices

config = Config(
    database_params=DatabaseParams(
        num_rows_entity_table_choices=Choices("range", [500, 1000]),
        num_rows_activity_table_choices=Choices("range", [5000, 10000]),
    ),
    schema_file=str(schema_path),  # pass the schema file to config
)

print(config)

/Users/vigneshk/plurel/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config(database_params=DatabaseParams(table_layout_choices=Choices(kind='set', value=['BarabasiAlbert', 'ReverseRandomTree', 'WattsStrogatz']), num_tables_choices=Choices(kind='range', value=[3, 20]), num_rows_entity_table_choices=Choices(kind='range', value=[500, 1000]), num_rows_activity_table_choices=Choices(kind='range', value=[5000, 10000]), num_cols_choices=Choices(kind='range', value=[3, 40]), min_timestamp=Timestamp('1990-01-01 00:00:00'), max_timestamp=Timestamp('2025-01-01 00:00:00'), column_nan_perc=Choices(kind='range', value=[0.01, 0.1])), scm_params=SCMParams(scm_layout_choices=Choices(kind='set', value=['ErdosRenyi', 'BarabasiAlbert', 'RandomTree', 'ReverseRandomTree', 'Layered']), scm_col_node_perc_choices=Choices(kind='range', value=[0.3, 0.9]), num_categories_choices=Choices(kind='range', value=[2, 10]), col_stype_choices=Choices(kind='set', value=[<stype.categorical: 'categorical'>, <stype.numerical: 'numerical'>]), initialization_choices=Choices(kind='set', value=[<

## 3. Generate one synthetic database

In [4]:
db = SyntheticDataset(seed=0, config=config).make_db()
db

Generating bi_fk_pk_graphs: 0it [00:00, ?it/s]
generating rows:   0%|          | 0/617 [00:00<?, ?it/s]/Users/vigneshk/plurel/plurel/utils.py:275: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4424.)
  sims = self.E.weight @ x.T
generating rows: 100%|██████████| 617/617 [00:00<00:00, 4203.00it/s]
adding edges in bi_hsbm: 7607it [00:10, 723.92it/s]0:00<?, ?it/s]
generating rows: 100%|██████████| 7607/7607 [00:04<00:00, 1893.45it/s]

Adding uniformly spaced timestamps from 2014-09-08 00:00:00 to 2017-04-08 00:00:00


Database()

## 4. Inspect generated tables

In [5]:
# Tables are returned as a mapping: table_name -> pandas.DataFrame (or equivalent table object)
tables = db.table_dict
list(tables.keys())

['orders', 'users']

In [7]:
# preview of each table
for name, table in tables.items():
    print(f"\n=== {name} ===")
    try:
        display(table.head())
    except Exception:
        # Fallback if `display` isn't available / table isn't a DataFrame
        print(table.df.head())


=== orders ===
   order_id  user_id    amount order_type                       date
0         0       27  0.450017     online 2014-09-08 00:00:00.000000
1         1      132  0.450033    instore 2014-09-08 02:58:31.964238
2         2       56  0.450052    instore 2014-09-08 05:57:03.928477
3         3      113  0.450071     online 2014-09-08 08:55:35.892716
4         4      127  0.450092    instore 2014-09-08 11:54:07.856955

=== users ===
   user_id  status
0        0  active
1        1  banned
2        2  active
3        3  active
4        4  active
